This notebook can be used to generate the ZY surface code that we used for our research.
This creates the generic template circuits with dummy probability values. I used these generic circuits and filled pre-calculated biased probability values through bash scripting.

In [5]:
import os
import sys

import shutil

sys.path.insert(0, 'functions') 
import oscAntPritgencircuitv2 as oscant
import morefuncs as morefuncs

# Update with any changes to these files:
from importlib import reload
reload(morefuncs)
reload(oscant)

<module 'oscAntPritgencircuitv2' from '/Users/priteshthakur/Documents/FRI/qec/oscAntPritgencircuitv2.py'>

In [ ]:
reload(morefuncs)
reload(oscant)


directory = "circuits/"

# Delete existing circuits (so as not to simulate them)
shutil.rmtree(f'{directory}/ZY/', ignore_errors=True)

# Make directories to store new circuits:
for path in ['ZY/y', 'ZY/z']:
    os.makedirs(f'{directory}/{path}', exist_ok=True)


# Generate circuits:

# x = (...) and z = (...) below define the CNOT order for y and z type stabilisers. E.g. order 10231203 avoids hook errors in the rotated code and is a worst-case CNOT order for the unrotated code if doing memory Z experiments (worst-case taken to be most applicable to all states). 

# I am setting this to only one p value to create generic template circuits. I used other bash scripts to generate the circuits using pre-calculated biased probability values.
p_values = [0.0005]        

# CNOT orders for y-type and z-type stabiliser measurements:
y = (1,0,2,3)
z = (1,2,0,3)

for ro in ['unro','ro']:
    
    if ro == 'unro':
        unro_CNOT_order = int(''.join(map(str, y + z)))
        ds = range(2,6)

    elif ro == 'ro':
        ro_CNOT_order = int(''.join(map(str, y + z)))
        ds = range(2,6)
    
    for d in ds:
        for p in p_values:
            for b in 'yz':
                thecircuit = oscant.generate_circuit(
                    f"surface_code:{ro}tated_memory_{'x' if b == 'y' else 'z'}",
                    rounds=3*d, 
                    distance=d, 
                    after_clifford_depolarization=p,
                    after_reset_flip_probability=p,
                    before_measure_flip_probability=p, 
                    x_cnot_order = y,
                    z_cnot_order = z,
                    exclude_other_basis_detectors = False, # include all stabiliser measurements | set it True if you want to exclude other basis measurements.
                )

                morefuncs.add_idling_errors_and_save_circuit(thecircuit, b, d, p, ro, y, z, directory)

ValueError: Failed to open circuits/temp_circuit.stim